# Medidas de actividad por electrodo: Distribuciones de prob de señal, desviaciones y modas.

Veamos. Franco dice que hay una distribucion trimodal en los datos. Cada moda indica la superposición de otra señal. Una es el ruido de fondo. La mas violenta debe ser la actividad real.

In [1]:
using MAT 
DatosAparato=matopen("propagacion.mat");
matread("propagacion.mat")

Dict{ASCIIString,Any} with 4 entries:
  "SamplingFrequency" => 7022.0
  "StopFrame"         => 5.549486e6
  "StartFrame"        => 5.545273e6
  "RawMatrix"         => [4125.0 2.01416015625 -28.1982421875 -10.07080078125 -…

In [2]:
LasMedicionesCrudas=read(DatosAparato, "RawMatrix");
tamano=size(LasMedicionesCrudas)

(64,64,4213)

In [3]:
function umbral(x)
    result=((abs(x)>200) ? 0 : x)
end;

function aplastar(x)
    result=((abs(x)<40) ? 0 : x)
end;


In [10]:
#quitemos el mugroso canal saturado y similares
DatosDeSaturados=map(umbral, LasMedicionesCrudas);

Saquemos estadistica sobre todos los datos y veamos cuanto se tarda.

In [11]:
DesviacionPorElectrodo=zeros(tamano[1],tamano[2])
for j=1:64, k=1:64
    DesviacionPorElectrodo[j,k]=std(reshape(LasMedicionesCrudas[j,k,:],tamano[3]));
end
writedlm("DesviacionesTotalesEjemplo01.dat", DesviacionPorElectrodo)


In [14]:
map!(x->abs(x)>60?0:x, DesviacionPorElectrodo)

64x64 Array{Float64,2}:
  0.0     13.572   16.0092  14.2284  …  16.4378  15.2936  14.4599  17.0095
 15.2189  15.3733  16.9369  16.2843     17.1325  16.9005  15.011   14.6944
 15.1487  16.205   15.4246  14.9731     14.803   15.7776  16.701   15.3513
 15.225   16.4475  15.4804  23.5391     16.4577  19.9773  16.1923  15.7709
 15.1806  15.9918  16.2133  15.5441     16.3465  18.5022  15.4268  15.3136
 15.863   14.8662  15.5449  15.7024  …  17.8495  15.2282  16.378   21.7898
 15.3221  16.1782  14.9934  15.3517     16.9116  16.6305  16.6185  17.2088
 15.3783  14.7717  16.7101  16.2186     15.6026  17.3575  18.1438  15.8534
 14.5119  15.5706  14.7123  14.7602     16.5219  15.6425  15.2475  14.5104
 15.8327  15.4433  18.3297  16.4969     17.7183  14.9865  15.221   13.3931
 15.5044  14.9394  16.1324  15.7822  …  17.9097  17.7534  15.8622  15.5514
 14.7791  15.1567  17.269   16.3143     16.7417  15.5708  15.8883  14.9204
 16.6736  15.5399  14.0674  15.2925     17.9801  17.1504  17.0267  16.8321
 

In [5]:
include ("Otsu01.jl")

OtsuUmbralizar (generic function with 1 method)

In [16]:
Mascara=OtsuUmbralizar(DesviacionPorElectrodo)
writedlm("MascaraOtsu01.dat", Mascara)

Ahora toca hacer un supuesto "smoothing" de los datos. Se me ocurre lo siguiente: promediar los datos de 50 en 50 frames y ver que pasa, moviendo la ventana, claro esta, paso por paso.

In [17]:
function SuavizarDatos(data, param)
    tantos=length(data)
    result=zeros(data)
    for j=1:(tantos-param-1)
        chusco=sum(data[j:j+param])/param
        result[j]=chusco
    end
    return result
end

GaussianKernel=readdlm("GaussianMatrixFilter01.dat");
function GaussianSmooth(Datos)
    tamano1=size(Datos)
    colchonhorizontal=zeros(tamano1[1],3)
    temp=cat(2, colchonhorizontal, Datos, colchonhorizontal)
    tamano2=size(temp)
    colchonvertical=zeros(3,tamano2[2])
    temp=cat(1, colchonvertical,temp,colchonvertical)
    result=zeros(Datos)
    for j=4:tamano1[1]+3, k=4:tamano1[2]+3
        #los indices van primero, "renglones", luego "columnas", etc
        aux=temp[j-3:j+3,k-3:k+3]
        result[j-3,k-3]=sum(GaussianKernel.*aux)
    end
    result=result*maximum(Datos)/maximum(result)
    return result
end

GaussianSmooth (generic function with 1 method)

In [20]:
MascaraSuave=GaussianSmooth(Mascara)
writedlm("MascaraSuave01.dat", MascaraSuave)

In [21]:
DobleSuave=GaussianSmooth(MascaraSuave);
writedlm("Mascara2vecesSuave01.dat", DobleSuave)

In [23]:
Apachurrada=map(x->x<7?0:x, MascaraSuave)
Apachurrada=GaussianSmooth(Apachurrada)
writedlm("MascaraApachurrada01.dat", Apachurrada)

In [8]:
function desviaporventanas(data, param)
    tantos=length(data)
    result=zeros(data)
    for j=1:(tantos-param-1)
        chusco=std(data[j:j+param])
        result[j]=chusco
    end
    return result
    
end
    

desviaporventanas (generic function with 1 method)

In [4]:
PromediosPorCanal=zeros(64,64);
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(LasMedicionesCrudas[j,k,:],4213)
        PromediosPorCanal[j,k]=mean(ChorizoExemplo)
    end
end


In [10]:
DatosCentrados=zeros(LasMedicionesCrudas);
for t=1:4213
    DatosCentrados[:,:,t]=LasMedicionesCrudas[:,:,t]-PromediosPorCanal
end

In [12]:
for j=1:64
    ChorizoExemplo=reshape(DatosCentrados[30,j,:],4213);
    chairo=hist(ChorizoExemplo, cajas)
    curamaca[j+1]=chairo[2]
end

In [5]:
Variancias=zeros(PromediosPorCanal)
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(LasMedicionesCrudas[j,k,:],4213)
        Variancias[j,k]=std(ChorizoExemplo)
    end
end


In [14]:
paescribir=reshape(Variancias, 4096);
writedlm("VarianciasCentradas.dat", paescribir)
writedlm("VarianciasCentradasTabla.dat", Variancias)

In [15]:
VarianciasVentaneadas=zeros(DatosCentrados)
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(DatosCentrados[j,k,:],4213)
        VarianciasVentaneadas[j,k,:]=desviaporventanas(ChorizoExemplo,50)
    end
end

In [16]:
zacatito=zeros(Float64,(64,64))
for t=1:4213
    zacatlan="CuadradosConSombra-$t.dat"
    zacatito=zacatito*0.5+VarianciasVentaneadas[:,:,t]
    writedlm(zacatlan,zacatito )
end

In [64]:
otrascajas=range(0,0.75,41)
promediovariancias=mean(paescribir);
chairo=hist(paescribir, otrascajas)
cruac=DataFrame();
cruac[1]=collect(otrascajas[1:end-1]);
cruac[2]=chairo[2];
writetable("HistoVariancias.dat", cruac, separator='\t')

In [30]:
minimum(paescribir)

12.096726356719277

In [63]:
sum(chairo[2])

4094

In [33]:
import Distributions

In [36]:
?Distributions

INFO: Loading help data...


Symbol not found. Falling back on apropos search ...
No help information found.


In [51]:
Distributions.fit(Distributions.Gamma, paescribir)

Gamma(α=18.51755901600754, β=0.9008965546617297)

In [57]:
Distributions.fit(Distributions.Beta, paescribir)

LoadError: α and β must be positive
while loading In[57], in expression starting on line 1

In [58]:
sum(chairo[2]/4079.)

1.0

In [68]:
paescribirunfolded=sort(paescribir);
paescribirunfolded=diff(paescribirunfolded); #checa el metodo diff

In [69]:
writedlm("DiferenciasVariancias01.dat", paescribirunfolded)

In [81]:
otrasbins=range(0,0.005,200)
histodiff=hist(paescribirunfolded,otrasbins)
cruac=DataFrame();
cruac[1]=collect(otrasbins[1:end-1]);
cruac[2]=histodiff[2];
writetable("HistoDiffVariancias.dat", cruac, separator='\t')

In [82]:
Distributions.fit(Distributions.Gamma, paescribirunfolded)

Gamma(α=0.12669302226161874, β=3.083200873659658)

In [83]:
sum(cruac[2])

4092

Okey, vamos a seguir Francos Way a partir de ahora. Veamos que endiabladas cosas esas funcionan mas o menos menos chungo

In [6]:
Mascara=zeros(Variancias)
Mascara=map(x->((x<26)&&(x>17))?1:0, Variancias);
writedlm("mascara.dat", Mascara)

In [90]:
?std

Base.std(v[, region])

   Compute the sample standard deviation of a vector or array "v",
   optionally along dimensions in "region". The algorithm returns an
   estimator of the generative distribution's standard deviation under
   the assumption that each entry of "v" is an IID drawn from that
   generative distribution. This computation is equivalent to
   calculating "sqrt(sum((v - mean(v)).^2) / (length(v) - 1))".
   Note: Julia does not ignore "NaN" values in the computation. For
   applications requiring the handling of missing data, the
   "DataArray" package is recommended.


In [91]:
std(LasMedicionesCrudas)

72.0767825266103